In [41]:
import pandas as pd
import numpy as np

In [42]:
data = pd.read_csv('train.csv')
# print(data.iloc[:,0:2]) #iloc to index the columns

# frac - specifies 80%
# random_state - it saves the current state of the split. Used for reproducability
# Spliting data int Train-Validation[80-20]
train = data.sample(frac=0.8, random_state=200) 
validation = data.drop(train.index)
categorical_train_data = train.iloc[:, 7:10]
categorical_validation_data = validation.iloc[:, 7:10]
print(categorical_validation_data)

             sales  salary  left
2       accounting     low     1
3            sales  medium     0
5        technical  medium     0
17           sales  medium     0
18           RandD     low     0
27       technical    high     0
28     product_mng  medium     0
37       marketing  medium     0
40           sales  medium     0
46       technical  medium     0
48       technical  medium     0
49           sales     low     0
50       technical     low     0
57         support    high     1
72       technical  medium     0
76       marketing    high     1
82           sales  medium     0
96      accounting     low     0
124        support     low     1
129      technical  medium     0
132      technical     low     0
137      technical     low     1
140      technical  medium     0
146          sales     low     0
151          sales     low     0
153      technical    high     0
160      marketing     low     1
169             IT     low     0
171      technical  medium     0
172       

In [46]:
def calc_entropy():
    target = categorical_train_data.keys()[-1] # target -> Labels
    entropy = 0
    values = categorical_train_data[target].unique() # Finds the unique values in target variable, i.e. [0, 1]
    for value in values:
        '''
            total count of 0's         total count of 1's
            -------------------   ,    ---------------------  = frac
            total numbers of rows      total number of rows
        '''
        frac = categorical_train_data[target].value_counts()[value]/len(categorical_train_data[target])
        entropy += -frac*np.log2(frac)
    return(entropy) # returns entropy of whole dataset

In [68]:
def calc_info_attribute(attribute):
    target = categorical_train_data.keys()[-1]
    variables = categorical_train_data[attribute].unique() # get unique variables in specified attribute
    target_variables = categorical_train_data[target].unique() # get unique in target, i.e. [0, 1]
    info = 0
    for v in variables:
        entropy = 0
        for target_v in target_variables:
            n = len(categorical_train_data[attribute][categorical_train_data[attribute] == v][categorical_train_data[target] == target_v])
            d = len(categorical_train_data[attribute][categorical_train_data[attribute] == v])
            frac1 = n/(d + np.finfo(float).eps) # eps - smallest representable number such that 1.0 + eps != 1.0
            entropy += -frac1 * np.log2(frac1)
        frac2 = d/len(categorical_train_data)
        info += -frac2 * entropy
    return abs(info)

0.7932274601977748

In [69]:
def winner_attribute():
    info_gain = []
    for attribute in categorical_train_data.keys()[:-1]:
        info_gain.append(calc_entropy - calc_info_attribute(attribute))
    return categorical_train_data.keys()[:-1][np.argmax(info_gain)]

In [ ]:
def build_tree():
    